In [648]:
import numpy as np

def rand(n, m):
    return np.random.random((n, m))

def bmat(blocks):
    return np.asarray(np.bmat(blocks))

def schur(Ainv, B, C, D):
    '''
    M = [[A, B],
         [C, D]]
    Minv = schur(Ainv, B, C, D)
    '''
    n, m = B.shape
    assert C.shape == (m, n) and D.shape == (m, m)

    # Apply Ainv to columns of B
    CAinvB = np.array([C.dot(Ainv(b)) for b in B.T]).T
    
    # inverse of Schur's Complement
    Sinv = np.linalg.inv( CAinvB - D )
        
    def Minv(f, g):
        λ = Sinv.dot(C.dot(Ainv(f)) - g)
        x = Ainv(f - B.dot(λ))
        return x, λ 
    
    return Minv

def schur_lambda(Ainv, B, C, D, n, m):
    '''
    Ainv, B, C, D, are all lambda expressions
    and the dimensions of the problem are (n+m, n+m)
    where presumably n>>m.
    '''
    
    # iterate over the columns of B
    CAinvB = np.array([C(Ainv(B(i))) for i in np.eye(m)]).T
    D = np.array([D(i) for i in np.eye(m)]).T

    # inverse of Schur's Complement
    Sinv = np.linalg.inv( CAinvB - D )

    def Minv(f, g):
        λ = Sinv.dot(C(Ainv(f)) - g)
        x = Ainv(f - B(λ))
        return x, λ 
    
    return Minv


In [649]:
n, m = 5, 2
np.random.seed(1)
f, g = rand(n, 1), rand(m, 1)
A = rand(n, n)
B = rand(n, m)
C = rand(m, n)
D = rand(m, m)
Ainv = np.linalg.inv(A)
M = bmat([[A, B],[C,D]])
Minv = np.linalg.inv(M)
λ = lambda A: lambda x: A.dot(x)

Mschur = schur(λ(Ainv), B, C, D)
assert np.allclose(Minv.dot(np.vstack([f,g])), np.vstack(Mschur(f, g)))

Mschur = schur_lambda(λ(Ainv), λ(B), λ(C), λ(D), n, m)
assert np.allclose(Minv.dot(np.vstack([f,g])), np.vstack(Mschur(f, g)))

#print([λ(Ainv)(b) for b in B.T])
#print([λ(Ainv)(λ(B)(i)) for i in np.eye(m)])
#print(D)
#print(np.array([λ(D)(i) for i in np.eye(m)]).T)

In [650]:
B = np.arange(n*m).reshape((n,m))
C = np.arange(n*m).reshape((m, n))
D = np.arange(m*m).reshape((m,m))
print(B)
print(C)
print(D)
print(B.dot(np.eye(B.shape[-1], B.shape[-1])))
print(C.dot(np.eye(C.shape[-1], C.shape[-1])))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[[0 1 2 3 4]
 [5 6 7 8 9]]
[[0 1]
 [2 3]]
[[ 0.  1.]
 [ 2.  3.]
 [ 4.  5.]
 [ 6.  7.]
 [ 8.  9.]]
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]]


In [651]:
from dec.spectral import I_diag

In [652]:
def fourier_sin(a, b):
    r'''
    Corresponds to :math:`f(x) \mapsto \int_{x+a}^{x+b} f(\xi) \sin(\xi) d\xi`
    '''

    def K(x):
        N = x.shape[0]
        x = np.array(x, dtype=np.complex)
        
        x = np.hstack([[0], x, [0]])
        x = (np.roll(x,+1) - np.roll(x,-1))
        x *= I_diag(N+2, a, b) /2j
        rslt = x[1:-1]

        rslt[ 0] += x[-1]
        rslt[-1] += x[0]
        return rslt
    
    def Kinv(x):
        # Make sure type is coerced to complex, otherwise numpy ignores the complex parts
        # and reverts to reals.
        x = np.array(x.copy(), dtype=np.complex)
        N = x.shape[0]
        I = I_diag(N+2, a, b)
        x /= I[1:-1]

        if (np.isclose(I[0], I[N]) or 
            np.isclose(I[1], I[N+1]) or 
            np.isclose(I[0]*I[1], I[N]*I[N+1])):
            raise ValueError("Singular operator.")

        y = np.zeros(N, dtype=np.complex)
        # The computations below are essentially Schur's complement?
        E = np.sum(x[::2]); O = np.sum(x[1::2])
        if N % 2 == 0:
            y[0]  = O/(1-I[0]/I[N])
            y[-1] = E/(I[N+1]/I[1]-1)
        else:
            y[0]  = (I[1]/I[N+1]*E+O)/(1-I[1]*I[0]/I[N]/I[N+1])
            y[-1] = (I[N]/I[0]*E+O)/(I[N]*I[N+1]/I[0]/I[1]-1)
        
        x[0]  -= y[-1]*I[N+1]/I[1]
        x[-1] -= -y[0]*I[0]/I[N]

        # This should be the crux of the inverse
        x = np.hstack([[-y[0]], x , [y[-1]]])
        y[::2] = -np.cumsum(x[::2])[:-1]
        y[1::2] = np.cumsum(x[1::2][::-1])[:-1][::-1]

        y *= 2j

        return y
    
    return K, Kinv

In [662]:
K, Kinv = fourier_sin(0, 0.1)
x = np.ones(4)
y = K(x)
assert np.allclose(x, Kinv(y))

In [663]:
N = 1000
x = K(np.ones(N))

In [664]:
%timeit K(np.ones(N))

10000 loops, best of 3: 192 µs per loop


In [665]:
%timeit Kinv(x)

1000 loops, best of 3: 303 µs per loop


In [666]:
K_, Kinv_ = fourier_sin_schur(0, 0.1)
%timeit K_(x)

1000 loops, best of 3: 199 µs per loop


In [658]:
import dec.spectral as sp

def Aa(x):
    f = np.hstack([ [-x[1]], x[:-2]-x[2:], [x[-2]] ])
    return f

print(to_matrix(Aa, 6))
#print(np.linalg.inv(to_matrix(Aa, 6)))

[[-0. -1. -0. -0. -0. -0.]
 [ 1.  0. -1.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.]
 [ 0.  0.  0.  1.  0. -1.]
 [ 0.  0.  0.  0.  1.  0.]]


In [667]:
def fourier_sin_schur(a, b):

    def get_ABCD(n, m):

        I = sp.I_diag(n+m+2, a, b) / 2j
        zeros = lambda n: np.zeros(n, dtype=np.complex)

        def A(x): 
            f = np.hstack([ [-x[1]], x[:-2]-x[2:], [x[-2]] ])
            f *= I[2:-2]
            return f
        
        def B(λ):
            f = zeros(n)
            f[0]  =  I[ 2]*λ[0]
            f[-1] = -I[-3]*λ[1]
            return f

        def C(x):
            g  = zeros(m)
            g[0] = -I[ 1]*x[ 0]
            g[1] =  I[-2]*x[-1]
            return g
        
        def D(λ):
            g = zeros(m)
            g[0] =  I[-1]*λ[1]
            g[1] = -I[ 0]*λ[0]
            return g
        
        return A, B, C, D

    def K(x):

        x = np.array(x, dtype=np.complex)
        
        x, λ = x[1:-1], x[[0, -1]]
        n = len(x)
        m = len(λ)

        A, B, C, D = get_ABCD(n, m)
        f = A(x) + B(λ)
        g = C(x) + D(λ)
        
        return np.hstack([ [g[0]], f, [g[1]] ])
    
    
    def Kinv(f):
        Minv = shur_lambda(A, B, C, D, n, m)
        return x
    
    return K, Kinv

K_, Kinv_ = fourier_sin(0, 1)
K, Kinv = fourier_sin_schur(0, 1)
X = np.arange(4)
Y = K_(X)
assert np.allclose(K(X), Y)
K(X).round(3), Y.round(3), Kinv_(Y).round(3)
K_(np.array([1,]))

array([ 0.45969769+0.j])

In [686]:
def I_diag_sy(N, a, b):
    from sympy import I, exp, Integer, symbol

    l = symbols(['I{}'.format(i) for i in range(N)])
#    l = []
#    for n in map(int, sp.freq(N)):
#         if n == 0:
#             l.append(b-a)
#         else:
#             n = Integer(n)
#             l.append( (exp(I*n*b) - exp(I*n*a))/(I*n) )/2/I
    return np.array(l)

def fourier_sin_schur_sym(a, b):
    from sympy import I, exp, Integer
        
    def K(x):
        N = x.shape[0]
        x = np.hstack([[0], x, [0]])

        x = (np.roll(x,+1) - np.roll(x,-1))
        x *= I_diag_sy(N+2, a, b)
        rslt = x[1:-1]

        rslt[ 0] += x[-1]
        rslt[-1] += x[0]
        return rslt

    def Kinv(x):
        pass
    
    return K, Kinv

a, b = symbols('a b')
print(I_diag_sy(3, a, b))
K, Kinv = fourier_sin_schur_sym(a, b)
X = np.array(sy.symbols('x0 x1 x2 x3 x4'))
K(X)

[I0 I1 I2]


array([-I1*x1 + I6*x4, I2*(x0 - x2), I3*(x1 - x3), I4*(x2 - x4),
       -I0*x0 + I5*x3], dtype=object)

In [676]:
from sympy import symbols
symbols(['I{}'.format(i) for i in range(5)])

[I0, I1, I2, I3, I4]